#### 아래 내용은 Intel® AI for Youth 프로그램을 참고하여
BrainAI와 BrainAI Coach Network에서 개발한 내용입니다.<br> 
상업적 사용은 불가하며 교육기관 및 학교에서 학생들 교육활동에 자유롭게 사용가능합니다.

# 프로젝트 제목: 교통 신호 인식 AI 모델링 및 모델 평가하기
자율주행자동차가 횡단보도에 정지 한 후 교통 신호를 판단하여 신호에 따라 주행할 수 있도록 하기 위해<br>
실시간 영상에서 관심 영역으로 지정된 교통 신호를 판단할 수 있도록 준비된 데이터로 AI 모델을 훈련하고<br>
훈련된 AI 모델이 교통 신호를 잘 인식하는 지 모델을 평가한다.

### Task 1. 필요 라이브러리 불러오기

In [1]:
import numpy as np
import cv2
import os
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, BatchNormalization, Conv2D, Activation, MaxPooling2D, Dropout, Flatten, Dense, Add
from tensorflow.keras.optimizers import Adam

### Task 2. 데이터 저장 폴더 불러오기

In [2]:
data_dir = './Data'
folders=[]
folders = [f for f in (os.listdir(data_dir))]

if '.ipynb_checkpoints' in folders:
    folders.remove('.ipynb_checkpoints')
print(folders)

['forward', 'left', 'noSign', 'stop', 'test']


### Task 3. 모델에 사용되는 이미지 크기 설정 및 데이터 분류 개수

In [3]:
input_height = 48
input_width = 48
input_channel = 3

input_shape = (input_height, input_width, input_channel)
num_classes = len(folders) - 1   

### Task 4. 훈련 셋, 테스트 셋, 레이블 데이터를 불러오기 

In [4]:
trainset = []
testset = []
label = []
test_files = []

In [5]:
for i in range(len(folders)):
    
    print("Loading images: " + folders[i])
    data_path = os.path.join(data_dir, str(folders[i]))
        
    for subdir, dirs, files in os.walk(data_path):
        for filename in files:
            file_path = data_path + os.sep + filename

            if file_path.endswith(".jpg") or file_path.endswith(".png"):
                image = cv2.imread(file_path)
                resized_image = cv2.resize(image,(input_width, input_height))

                if folders[i] == "test":
                    testset.append(resized_image)
                    test_files.append(file_path)

                else: 
                    trainset.append(resized_image)
                    
                    label_data = np.zeros(shape = (num_classes)) 
                    label_data[i] = 1.0
                    label.append(label_data)           

trainset = np.array(trainset)
testset = np.array(testset)
label = np.array(label)
print("# of Training Images: ", trainset.shape[0])
print("# of Test Images: ", testset.shape[0])

Loading images: forward
Loading images: left
Loading images: noSign
Loading images: stop
Loading images: test
# of Training Images:  871
# of Test Images:  40


### Task 5. 훈련데이터 및 평가데이터 정규화(0과 1 사이의 수로 변환)

In [6]:
trainset = trainset / 255.0
testset = testset / 255.0

In [7]:
folders.remove('test')

### Task 6. 모델 구성하기

In [8]:
model = Sequential()

model.add(Conv2D(64, kernel_size=3, 
                 activation='relu', 
                 kernel_initializer='he_normal', 
                 padding="valid",
                 input_shape=input_shape))

model.add(MaxPooling2D(2))
model.add(Dropout(0.3))

model.add(Conv2D(128, kernel_size=3, 
                 activation='relu', 
                 kernel_initializer='he_normal', 
                 padding='valid'))

model.add(MaxPooling2D(2))
model.add(Dropout(0.3))
model.add(Conv2D(256, kernel_size=3, kernel_initializer='he_normal', 
                 activation='relu'))

model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(256, kernel_initializer='he_normal', activation='relu'))
model.add(Dense(512, kernel_initializer='he_normal', activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 46, 46, 64)        1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 23, 23, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 21, 21, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 128)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 10, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 256)         2

### Task 7. AI 모델 최적 학습 방법 설계 및 모델 훈련

In [10]:
# 인공지능 모델을 이용한 최적 데이터 학습 방법 설계
adam=Adam()
model.compile(
    optimizer=adam,
    loss='categorical_crossentropy',
    metrics=['accuracy'])

# 인공지능 모델을 이용한 데이터 학습 실행
history = model.fit(
    trainset, label, 
    batch_size=20, 
    epochs= 10,  
    validation_split=0.005)

Epoch 1/10
44/44 [==============================] - 5s 111ms/step - loss: 1.4963 - accuracy: 0.7067 - val_loss: 0.0018 - val_accuracy: 1.0000
Epoch 2/10
44/44 [==============================] - 5s 106ms/step - loss: 0.0355 - accuracy: 0.9873 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/10
44/44 [==============================] - 5s 105ms/step - loss: 0.0242 - accuracy: 0.9896 - val_loss: 7.1526e-08 - val_accuracy: 1.0000
Epoch 4/10
44/44 [==============================] - 5s 105ms/step - loss: 0.0371 - accuracy: 0.9873 - val_loss: 9.5367e-08 - val_accuracy: 1.0000
Epoch 5/10
44/44 [==============================] - 5s 106ms/step - loss: 0.0428 - accuracy: 0.9896 - val_loss: 1.6689e-06 - val_accuracy: 1.0000
Epoch 6/10
44/44 [==============================] - 5s 105ms/step - loss: 0.0192 - accuracy: 0.9942 - val_loss: 5.7529e-05 - val_accuracy: 1.0000
Epoch 7/10
44/44 [==============================] - 5s 106ms/step - loss: 0.0103 - accuracy: 0.9977 - val_loss: 1.1921e-07 - val

### Task 8. 모델 저장

In [11]:
model_desc = model.to_json()
with open('./Model/model.json', 'w') as file_model:
    file_model.write(model_desc)
model.save_weights('./Model/weights.h5')

with open("./Model/labels.txt", "w") as txt_file:
    for line in folders:
        txt_file.write(line + "\n") 

### Task 9. 평가데이터를 이용한 모델 평가

In [12]:
# 평가데이터를 이용한 예측 정확도 확인
if testset.shape[0] != 0:
    prediction = model.predict(testset)
    result_predict = np.argmax(prediction, axis=1)
else:
    result_sparse = list()
print('File name\t  forecast category')

# 평가 데이터 파일명에 따른 예측 결과 값 출력
for file, label_id in zip(test_files, result_predict):
    filename = os.path.basename(file)
    label_name = folders[label_id] 
    print("{:} {: >13}".format(filename, label_name))

File name	  forecast category
forward (1).jpg       forward
forward (10).jpg       forward
forward (2).jpg       forward
forward (3).jpg       forward
forward (4).jpg       forward
forward (5).jpg       forward
forward (6).jpg       forward
forward (7).jpg       forward
forward (8).jpg       forward
forward (9).jpg       forward
left (1).jpg          left
left (10).jpg          left
left (2).jpg          left
left (3).jpg          left
left (4).jpg          left
left (5).jpg          left
left (6).jpg          left
left (7).jpg          left
left (8).jpg          left
left (9).jpg          left
noSign (1).jpg        noSign
noSign (10).jpg        noSign
noSign (2).jpg        noSign
noSign (3).jpg        noSign
noSign (4).jpg        noSign
noSign (5).jpg        noSign
noSign (6).jpg        noSign
noSign (7).jpg        noSign
noSign (8).jpg        noSign
noSign (9).jpg        noSign
stop (1).jpg          stop
stop (10).jpg          stop
stop (2).jpg          stop
stop (3).jpg          sto